# Introdução à Inteligência Artificial (2022/2023)

## Planeamento

### Conteúdos

* Afirmações lógicas
* Classe PlanningProblem
    * o conhecimento base
    * o objetivo
    * a classe Action
* Exemplos de formulação de problemas:
    * Torre com 3 blocos
    * Viagem à Roménia
    * Comer bolo
* Procura em espaço de estados
    * Procura progressiva (Forward)
* Exercícios


## Introdução

Nesta aula vamos formular problemas de planeamento usando a linguagem Python. Como o planeamento combina duas áreas de IA: procura e lógica, é necessário primeiro perceber como se escrevem afirmações lógicas em Python, antes de começarmos a formular problemas de planeamento.


### Recursos necessários
Para este guião os seguintes módulos são necessários (distribuídos juntamento com o guião):
* `planningPlus.py` - módulo principal
* `search.py` - módulo principal
* `logic.py` - módulo auxiliar
* `utils.py` - módulo auxiliar

### Convenções sintáticas

A utilização deste planeador de Python necessita de algumas convenções, para se definirem predicados, termos, e variáveis. Assim,

* Inicial maiúscula para termos e predicados (p.ex., Table, Block(A), On(A,Table), ...)
* Inicial minúscula para variáveis (p.ex., x, y, a, ...)

## Módulo *planningPlus.py* - breve explicação

Este módulo é uma variante muito ligeira do *planning.py* que está disponível no repositório aima-python, que contém a implementação (em Python) da generalidade dos algoritmos descritos no livro da disciplina (Russel & Norvig). Muitas das definições deste módulo não serão utilizadas. Vamos apenas concentrar-nos em algumas das suas classes e funções. No essencial, é disponibilizado o seguinte:

* A classe **PlanningProblem** que vamos utilizar para definir problemas de planeamento.
* A classe **Action** que é usada para representar um esquema de ação.
* As classes **ForwardPlan** e **BackwardPlan** para definir problemas de espaços de estados a serem usados nos algoritmos de procura.

Para a parte dedicada à formulação, apenas precisamos de usar as classes **PlanningProblem** e **Action** do módulo, que tem de ser importado, mas não deverão alterá-lo.

In [1]:
from planningPlus import *
from utils import *
from search import *

## O problema da torre com 3 blocos

O dominio deste problema consiste num conjunto de blocos em cima de uma mesa. Os blocos podem ser empilhados, mas apenas um bloco pode ser colocado diretamente em cima de outro. Um robot pode pegar num bloco e movê-lo para outra posição, em cima da mesa ou em cima de outro bloco. O robot apenas pode pegar num bloco de cada vez, o que significa que não pode pegar num bloco que tenha outro em cima. O objetivo é construir uma torre de blocos. A configuração que iremos usar neste exemplo também é conhecida como Anomalia de Sussman.

<img src="3-blocks.png" width="300">

Pretendemos formular este problema como um problema de planeamento usando PDDL (*Planning Domain Definition Language*). Para definirmos um problema temos de ter:

* um conhecimento base
* um conjunto de objetivos
* um conjunto de esquemas de ações

Cada conhecimento base é representado por um conjunto de expressões que descrevem o estado inicial do problema. Essa descrição é feita recorrendo a proposições lógicas de primeira ordem. A representação dos estados é cuidadosamente projetada para que um estado possa ser tratado como uma conjunção de expressões, que pode ser manipulada por inferência lógica, ou como um conjunto de expressões, que podem ser manipuladas com operações definidas.

Antes de descrevermos o nosso problema dos blocos, precisamos de perceber como trabalhar com expressões lógicas em Python. 

## Afirmações lógicas

Façamos uma revisão sobre como representamos em python, as expressões lógicas.

A classe `Expr` é usada para representar qualquer tipo de expressão lógica. O tipo mais simples de `Expr` que se pode definir usa a função `Symbol`:

In [2]:
Symbol('X')

X

Ou pode-se definir múltiplos simbolos ao mesmo tempo com a função `symbols`:

In [3]:
(X, Y, P, Q, F) = symbols('X, Y, P, Q, F')

Pode-se combinar objetos `Expr` com os operadores de Python (&, ~, |, ...). Se quisessemos formar a afirmação "P e não Q", seria da seguinte forma. Repare que P e Q são simbolos definidos anteriormente.

In [4]:
P & ~Q

(P & ~Q)

Isto funciona porque a classe `Expr` sobrepõe o operador `&` (*overloading*) com a definição:
```python
def __and__(self, other): return Expr('&',  self, other)
```
e faz sobreposições semelhantes para outros operadores. Um objeto `Expr` tem dois atributos:

* `op` para o operador, que é sempre uma string
* `args` para os argumentos, que é um tuplo com zero ou mais expressões

Por "expressões" entende-se uma instância de `Expr`, ou um número. 

Se não definirmos os simbolos não é possivel usar os operadores de Python para escrever expressões diretamente.

In [5]:
A & B

NameError: name 'A' is not defined

Vamos verificar os atributos de alguns exemplos de objetos `Expr`:

In [6]:
sentence = P & ~Q
sentence.op

'&'

In [7]:
sentence.args

(P, ~Q)

In [8]:
P.op

'P'

In [9]:
P.args

()

Também se podem definir predicados, como por exemplo

In [10]:
Pxy = P(X, Y)
Pxy.op

'P'

In [11]:
Pxy.args

(X, Y)

É importante salientar que a classe `Expr` apenas fornece uma forma de "representar" expressões. Cada um dos `args` num objeto `Expr` pode ser um símbolo, um número, ou outra `Expr`. Vejemos um exemplo onde usamos números.

Eis a tabela de operadores que podem ser usados para criar afirmações. Mas existe um problema: queremos usar os operadores de Python para construir afirmações, mas o Python não permite a seta de implicação como operador. Assim, teremos de usar uma notação ligeiramente diferente que o Python permita: `|'==>'|` em vez de apenas `==>`. Em alternativa, pode-se usar a forma mais verbosa do construtor de `Expr`:

| Operação                 | Livro | Input Python | Output Python | Input `Expr`
|--------------------------|----------------------|-------------------------|---|---|
| Negação                 | &not; P      | `~P`                       | `~P` | `Expr('~', P)`
| E                      | P &and; Q       | `P & Q`                     | `P & Q` | `Expr('&', P, Q)`
| Ou                       | P &or; Q | `P`<tt> &#124; </tt>`Q`| `P`<tt> &#124; </tt>`Q` | `Expr('`&#124;`', P, Q)`
| Desigualdade         | P &ne; Q     | `P ^ Q`                | `P ^ Q`  | `Expr('^', P, Q)`
| Implicação                  | P &rarr; Q    | `P` <tt>&#124;</tt>`'==>'`<tt>&#124;</tt> `Q`   | `P ==> Q` | `Expr('==>', P, Q)`
| Implicação contrária      | Q &larr; P     | `Q` <tt>&#124;</tt>`'<=='`<tt>&#124;</tt> `P`  |`Q <== P` | `Expr('<==', Q, P)`
| Equivalência            | P &harr; Q   | `P` <tt>&#124;</tt>`'<=>'`<tt>&#124;</tt> `Q`   |`P <=> Q` | `Expr('<=>', P, Q)`

In [12]:
~(P & Q)  |'==>'|  (~P | ~Q)

(~(P & Q) ==> (~P | ~Q))

Equivalentemente,

In [13]:
Expr('==>', ~(P & Q), ~P | ~Q)

(~(P & Q) ==> (~P | ~Q))

Existe uma forma mais simples de definir afirmações, usando a função `expr`. Esta função tem como input uma string, e faz "parsing" para um objeto `Expr`. A string pode conter os operadores setas: `==>`, `<==`, ou `<==>`, que serão tratados como se fossem operadores de Python. Além disso, `expr` define automaticamente quaisquer dos respectivos simbolos, não é necessário serem pré-definidos (como fizemos com o P e Q).

In [14]:
expr('~(P & Q) ==> (~P | ~Q)')

(~(P & Q) ==> (~P | ~Q))

### Para além do universo proposicional

Se quisermos escrever a expressão "O Lápis está sobre a mesa", usamos predicados da seguinte forma

In [15]:
f = expr('Sobre(Lapis,Mesa)')

O operador e os argumentos são respetivamente

In [16]:
f.op

'Sobre'

In [17]:
f.args

(Lapis, Mesa)

Mais alguns exemplos de predicados que se podem escrever:

In [18]:
# O disco está no pino
expr('Esta(Disco, Pino)')

Esta(Disco, Pino)

In [19]:
# O supermercado (SM) Vende laranjas
expr('Vende(SM, Laranjas)')

Vende(SM, Laranjas)

In [20]:
# 1 precede o 2
expr('Precede(1,2)')

Precede(1, 2)

In [21]:
# 2 não precede o 1
expr('~Precede(2,1)')

~Precede(2, 1)

## A classe *PlanningProblem*

A classe `PlanningProblem` é usada para representar problemas de planeamento. Os seguintes atributos são essenciais para se conseguir definir um problema:

* um conhecimento base (*initial*)
* objetivos (*goals*)
* conjunto de esquemas de ações viáveis (*actions*)
* o domínio do problema (*domain*)

A seguir apresenta-se o construtor desta classe:

```python
def __init__(self, initial, goals, actions, domain=None):
        self.initial = self.convert(initial) if domain is None else self.convert(initial) + self.convert(domain)
        self.goals = self.convert(goals)
        self.actions = actions
        self.domain = domain
```

##### O domínio

O atributo `domain` do problema é um conjunto de expressões que descrevem a informação estática, i.e., contém informação sobre os tipos, propriedades e relações entre os objetos representados, informação essa que não é alterada com as acções, e também não será alterada durante o processo de procura da solução.

Vamos tentar descrever o domínio para o problema dos blocos. Para tal é necessário criar predicados que descrevam os objetos.
<img src="3-blocks.png" width="300">

Sabemos que temos três blocos (designemo-los por A, B e C) e uma mesa. Então podemos definir o predicado formado por um argumento e o operador 'Bloco', que indica que um determinado objecto é do tiplo bloco. Por exemplo, o domínio irá conter a expressão seguinte, para o bloco A:

In [22]:
expr('Bloco(A)')

Bloco(A)

Apenas estamos a considerar os três blocos para o domínio e não consideramos a mesa. Aqui poderíamos ter três alternativas:
* dizer que Mesa é mesa (p.ex., `Mesa(Mesa)`), 
* dizer que Table não é bloco (`~Bloco(Mesa)`), 
* ou então não dizer nada. 

Apresentamos o terceiro caso, pois esse objeto não irá sofrer alteração nenhuma durante a aplicação das ações, i.e., nunca usaremos `Mesa(x)` ou `~Bloco(x)` nas pré-condições, ou no domínio ou nos efeitos dos esquemas de ações. Ao contrário dos blocos que irão ser colocados em cima da mesa ou em cima de outro bloco. De facto, precisaremos apenas de restringir as variáveis das ações apenas para os blocos, e assim exigiremos que algumas variáveis do tipo `x` satisfaçam `Bloco(x)` e não serão instanciados para `x=Table`. Se tivessemos mais do que uma mesa e pudessemos transferir blocos entre mesas, então daria jeito termos `Mesa(x)`.

Assim, o domínio deste problema é dado por:

In [23]:
expr('Bloco(A) & Bloco(B) & Bloco(C)')

((Bloco(A) & Bloco(B)) & Bloco(C))

##### O conhecimento base

O atributo `initial` da classe `PlanningProblem`é uma conjunção de expressões que descrevem o conhecimento base do problema. Em qualquer situação, se uma expressão não foi previamente afirmada, então não é conhecida.

Vamos tentar descrever o conhecimento base para o problema dos blocos. Para tal é necessário criar operadores que descrevam o ambiente. Por exemplo, sabemos que o bloco A está em cima da mesa ("A sobre Mesa"), e então podemos descrever esta situação usando a seguinte expressão:

In [24]:
expr('Sobre(A, Mesa)')

Sobre(A, Mesa)

Para que o robô possa pegar num bloco é necessário sabermos que o bloco que se pega não tem nenhum outro por cima. O mesmo acontece quando se quer colocar um bloco em cima de outro, este último tem de estar livre. Vamos arranjar um predicado com o operador `Livre` e um argumento para representar essa informação. Na situação inicial, sabemos que por cima do bloco C não existe nada, que o podemos representar pela expressão:

In [25]:
expr('Livre(C)')

Livre(C)

Para este problema, consideramos que a mesa (Mesa) é infinita, e haverá sempre espaço para lá colocar mais um bloco, mesmo que tivessemos uma centena de blocos. Assim, é desnecessário estar a afirmar `Livre(Mesa)`. Poderíamos fazê-lo mas por uma questão de parsimónia (reduzir ao máximo o que é redundante) não o fazemos.

Assim o conhecimento base do problema (ver figura) pode ser definido diretamente, sem necessitar de previamente terem sido definidos estes predicados, como sendo:

In [26]:
expr('Sobre(A, Mesa) & Sobre(B, Mesa) & On(C, A) & Livre(B) & Livre(C)')

((((Sobre(A, Mesa) & Sobre(B, Mesa)) & On(C, A)) & Livre(B)) & Livre(C))

##### Exercício 1
Como exprimiria o conhecimento de uma situação em que temos os mesmos 3 blocos, mas cada um deles sobre a mesa?

In [27]:
### Solução do exercício 1

expr('Sobre(A, Mesa) & Sobre(B, Mesa) & Sobre(C, Mesa) & Livre(A) & Livre(B) & Livre(C)')

(((((Sobre(A, Mesa) & Sobre(B, Mesa)) & Sobre(C, Mesa)) & Livre(A)) & Livre(B)) & Livre(C))

##### O objetivo

O atributo `goals` de `PlanningProblem` é uma expressão ou conjunção de expressões (positivas ou negativas) que indicam os objetivos a atingir no problema. No caso deste problema dos blocos, queremos ter o A por cima do B, o B por cima do C e C em cima da mesa. Assim, queremos a expressão

In [28]:
expr('Sobre(B, C) & Sobre(A, B)')

(Sobre(B, C) & Sobre(A, B))

Notem também que não temos `Sobre(C,Table)`. Poderíamos fazê-lo mas não parece ser fundamental. Existindo 3 blocos apenas, em que B fica em cima de C e A em cima de B, necessariamente C terá de ficar na mesa, sem termos de o exprimir. Por isso foi omitido do objectivo.

Notem que poderíamos também incluir `Livre(A)`na conjunção, mas não é de novo fundamental. A questão é que pelo facto de existirem apenas 3 blocos, A necessariamente ficará livre quando estiver sobre B e quando este está sobre C, e esse predicado será atingido quando os dois da conjunção em cima forem obtidos.

### Exercício 2
Exprimam um novo objetivo do problema dos blocos, considerando que apenas queremos que o bloco B passe a ter um outro bloco por cima.

In [29]:
# Solução do exercício 2

expr('~Livre(B)')

~Livre(B)

### Exercício 3
Exprimam um novo objetivo do problema dos blocos, considerando que apenas queremos que o bloco B esteja em cima de qualquer outro bloco.

In [30]:
# Solução do exercício 3
# Basta-nos dizer que B não está sobre a mesa e como não pode estar suspenso, estará que estar em cima de algum bloco

expr('~Sobre(B,Mesa)')

~Sobre(B, Mesa)

##### Esquemas de ações

O atributo `actions` de `PlanningProblem` contém uma lista de objetos da classe `Action` que representam esquemas de ações que posteriormente podem ser instanciadas e executadas no espaço de procura do problema. Uma instância da classe `Action` é definida pelo nome da ação, uma lista de variáveis usadas na ação, pré-condições, efeitos e domínio. Vamos olhar para o construtor desta classe:

```python
def __init__(self, action, precond, effect, domain=None):
        if isinstance(action, str):
            action = expr(action)
        self.name = action.op
        self.args = action.args
        self.precond = self.convert(precond) if domain is None else self.convert(precond) + self.convert(domain)
        self.effect = self.convert(effect)
        self.domain = domain
```

Esta classe representa um esquema de ação que inclui uma expressão, um domínio, as pré-condições, e os efeitos após a ação ser aplicada. 
* Os atributos *name* e *args* provêm do argumento *action* do construtor, o qual é uma instância de `Expr`, ou seja, tem um operador e argumentos (como visto anteriormente). 
* O atributo *precond* é uma conjunção de expressões (positivas ou negativas) com as pré-condições que devem ser verificadas no estado para se poder aplicar a ação.
* o atributo *effect* é uma conjunção de expressões com os efeitos/alterações ao estado após a aplicação da ação.
* O atributo *domain* descreve a informação estática do problema associada a uma ação. Evita aumentar as pré-condições com conhecimento estático. A definição deste atributo irá ser importante para a eficiência dos planeadores quando têm ações com variáveis para filtrar as permutações dos objetos do problema que podem gerar regras concretas a partir das regras genéricas.

As condições e efeitos negativos são introduzidos usando o simbolo `~` antes da expressão, que internamente é codificado com um `Not` e colocado como prefixo da expressão. Por exemplo, a negação de `Em(obj, loc)` será introduzido como `~Em(obj, loc)` e internamente representado por `NotEm(obj, loc)`. O método `convert` da classe `Action`, para além de converter as negações, pega numa string e faz "parsing", removendo as conjunções (se existirem) e retorna uma lista de objetos `Expr`.

Vamos ilustrar a execução dos efeitos através de um exemplo. Se nos efeitos de uma acção já innstanciada tivermos `NotEm(O, L)`, retira-se o `Em(O, L)` e insere-se o `NotEm(O, L)`. No caso, de termos um efeito positivo, por exemplo, `Em(O, L)` retira-se o `NotEm(O, L)` se existir e insere-se o `Em(O, L)`

Ainda nesta classe, existem outros métodos, incluindo o `check_precond`. Este método verifica se as pré-condições para uma dada ação são válidas, dado um estado. 

**IMPORTANTE** É preciso salientar que **a variáveis diferentes correspondem sempre a objetos diferentes**, não sendo necessário incluir expressões do género `b != x` nas pré-condições.

### Esquemas de acções no Problema dos Blocos
Vamos então tentar descrever os esquemas de ações do nosso problema dos blocos. Quando descrevemos um esquema de ação, descrevemos de forma genérica, usando variáveis, para que a ação possa ser aplicada a qualquer um dos objetos do problema. Por isso se chamam de esquemas: podem ser instanciados em várias acções concretas. Neste problema existem dois tipos de ações: `MoveParaMesa` e `Move`.

* **`MoveParaMesa(b, x)`** -- mover o bloco `b` que está em cima de `x` para a mesa.
    * Este esquema de acção apenas se aplica a dois blocos porque não pretendemos mover um objeto que está em cima da mesa para a mesa, nem queremos mover uma mesa que esteja em cima de alguma coisa para a mesa, o que significa que nem `b` nem `x` podem ser mesa, terão de ser blocos. 
    * nas pré-condições é obrigatório que não exista nada em cima do bloco `b` e é preciso que `b` esteja sobre `x`.
    * nas pós-condições sabemos que o bloco `x` passará a estar livre e que `b` passará a estar sobre a mesa e deixará de estar sobre `x`.
* **`Move(b, x, y)`** -- mover o bloco `b` que está em `x` para cima de `y`. Neste esquema queremos mover um bloco que pode estar em cima de outro ou em cima da mesa, para cima de outro bloco. 
    * Assim, no domínio não se coloca `Bloco(x)` porque `x` pode ser a mesa, mas exige-se `Bloco(b)` e `Bloco(y)`.
    * Nas pré-condições é necessário desde que ambos `b` e `y` não tenham nada por cima.
    * Nas pós-condições teremos que `x` passará a estar livre, que `y` deixará de estar livre e que `b` passará a estar em cima de `y` e não em cima de `x`.
    
Os domínios das acções levam a que as ações instanciadas com `MoveParaMesa(Mesa, x)`, `MoveParaMesa(x, Mesa)`, `Move(Mesa, x, y)` e `Move(x, y, Mesa)`, em que `x` e `y` são blocos A, B, C ou Mesa, nunca serão geradas no processo de planeamento.

Assim o esquema de ações será uma lista de objetos `Action`, um objeto para cada esquema de ação que se queira definir.

In [31]:
acoes = [Action('MoveParaMesa(b, x)',
                precond='Sobre(b, x) & Livre(b)',
                effect='Sobre(b, Mesa) & Livre(x) & ~Sobre(b, x)',
                domain='Bloco(b) & Bloco(x)'),
         Action('Move(b, x, y)',
                precond='Sobre(b, x) & Livre(b) & Livre(y)',
                effect='Sobre(b, y) & Livre(x) & ~Sobre(b, x) & ~Livre(y)',
                domain='Bloco(b) & Bloco(y)')]
print(acoes)

[MoveParaMesa(b, x), Move(b, x, y)]


### Formular o problema dos blocos

Vamos então formular o problema dos blocos, usando tudo o que aprendemos. Para o conhecimento base e o objetivo, não iremos usar o `expr` porque a classe `PlanningProblem` (tal como a classe `Action`) possui um método `convert` para converter strings em objetos `Expr`.

In [32]:
# conhecimento base
kb = 'Sobre(A, Mesa) & Sobre(B, Mesa) & Sobre(C, A) & Livre(B) & Livre(C)'
# dominio
dominio = 'Bloco(A) & Bloco(B) & Bloco(C)'
# objetivo
goal = 'Sobre(B, C) & Sobre(A, B)'

torre3blocos = PlanningProblem(initial=kb, goals=goal, actions=acoes, domain=dominio)

Olhemos para o estado inicial

In [33]:
print(torre3blocos.initial)

[Bloco(A), Bloco(B), Bloco(C), Livre(B), Livre(C), Sobre(A, Mesa), Sobre(B, Mesa), Sobre(C, A)]


Antes de aplicar alguma ação a este problema, vamos verificar se este problema atingiu o objetivo:

In [34]:
torre3blocos.goal_test()

False

Como se pode verificar, ainda não atingiu o objetivo. Vamos ver quais os esquemas de ações que existem:

In [35]:
for a in torre3blocos.actions:
    print(a)

MoveParaMesa(b, x)
Move(b, x, y)


### Instanciação dos esquemas de acções no Mundo dos Blocos
Apenas representámos esquemas de acções que são parametrizáveis com elementos que têm de pertencer aos domínios descritos.
Esses esquemas são descritos através de símbolos minùsculos que representam variáveis e são geradores de acções concretas que envolvem os elementos concretos do mundo que queremos representar, nmeste caso A, B, C e a Mesa.

Vamos então instanciar os esquemas de acções, usando o método `expand_actions` da classe `Planning_Problem`. Reparem que o campo `domain` na definição dos esquemas tem um papel fundamental na concretização das acções.
Estas são todas as acções possíveis dados os 3 blocos

In [36]:
torre3blocos.expand_actions()

[MoveParaMesa(B, C),
 MoveParaMesa(B, A),
 MoveParaMesa(C, B),
 MoveParaMesa(C, A),
 MoveParaMesa(A, B),
 MoveParaMesa(A, C),
 Move(B, C, A),
 Move(B, A, C),
 Move(B, Mesa, C),
 Move(B, Mesa, A),
 Move(C, B, A),
 Move(C, A, B),
 Move(C, Mesa, B),
 Move(C, Mesa, A),
 Move(A, B, C),
 Move(A, C, B),
 Move(A, Mesa, B),
 Move(A, Mesa, C)]

Importa sublinhar a importância do domínio do planeador e a sua interacção com o domínio do problema. Reparem que só as expressões concretizadas do domínio do problema que envolvem os termos dos próprios blocos é que satisfazem os domínios dos 2 esquemas de acções dados em termos de variáveis.

Na prática, existem quatro objetos definidos: A, B, C e Mesa, e todas as permutações destes objetos irão ser testadas em cada um dos esquemas de ações. No entanto, apenas algumas ações serão válidas por causa da filtragem do domínio, que só aceita alguns objetos como sendo blocos.

Notem que as ligações entre os objectos do mundo e as variáveis (*bindings*), na permutação, e os parâmetros da acção, são primeiro filtradas pelos domínios e se os satisfizerem então são propagadas tanto para as pré-condições como para os efeitos.

Vamos expandir as acções de novo com o modo verboso ligado, em que mostramos tanto as tentativas de instanciação bem sucedidas como as falhadas:

In [37]:
torre3blocos.expand_actions(verbose=True)

((Bloco(b) & Bloco(x)) ==> MoveParaMesa(b, x))
((Bloco(b) & Bloco(y)) ==> Move(b, x, y))
Objectos: {B, C, A, Mesa}

Generating concrete actions:

SCHEME = MoveParaMesa(b, x) with args = (b, x)

Remember, the action scheme is MoveParaMesa(b, x) and permutation = (B, C)
bindings = {b: B, x: C} -empty set? False
after propagating bindings in the action header = MoveParaMesa(B, C)
Lets propagate those bindings towards the preconditions
preconds: [Sobre(B, C), Livre(B), Bloco(B), Bloco(C)]
and also propagate them to the effects
effects: [Sobre(B, Mesa), Livre(C), NotSobre(B, C)]
In fact we have created a new concrete action: (MoveParaMesa(B, C), [Sobre(B, C), Livre(B), Bloco(B), Bloco(C)], [Sobre(B, Mesa), Livre(C), NotSobre(B, C)])

Remember, the action scheme is MoveParaMesa(b, x) and permutation = (B, A)
bindings = {b: B, x: A} -empty set? False
after propagating bindings in the action header = MoveParaMesa(B, A)
Lets propagate those bindings towards the preconditions
preconds: [Sobre(B,

[MoveParaMesa(B, C),
 MoveParaMesa(B, A),
 MoveParaMesa(C, B),
 MoveParaMesa(C, A),
 MoveParaMesa(A, B),
 MoveParaMesa(A, C),
 Move(B, C, A),
 Move(B, A, C),
 Move(B, Mesa, C),
 Move(B, Mesa, A),
 Move(C, B, A),
 Move(C, A, B),
 Move(C, Mesa, B),
 Move(C, Mesa, A),
 Move(A, B, C),
 Move(A, C, B),
 Move(A, Mesa, B),
 Move(A, Mesa, C)]

Se omitirmos o bloco C no domínio do problema, como apresentamos a seguir:

In [38]:
# conhecimento base
kb = 'Sobre(A, Mesa) & Sobre(B, Mesa) & Sobre(C, A) & Livre(B) & Livre(C)'
# dominio
dominio = 'Bloco(A) & Bloco(B)'
# objetivo
goal = 'Sobre(B, C) & Sobre(A, B)'

torre3blocos = PlanningProblem(initial=kb, goals=goal, actions=acoes, domain=dominio)

ao expandirmos as acções, fazendo

In [39]:
torre3blocos.expand_actions()

[MoveParaMesa(B, A),
 MoveParaMesa(A, B),
 Move(B, C, A),
 Move(B, Mesa, A),
 Move(A, C, B),
 Move(A, Mesa, B)]

obtivemos muitos menos acções. Afinal, desapareceram todas que impliquem mover o bloco C ou mover para cima do bloco C, porque C deixou de ser considerado como bloco ao desaparecer `Bloco(C)` do domínio do problema.

E se tivermos um domínio mal definido nas acções, podemos ter um conjunto de acções que não fazem sentido. 

Por exemplo, retiremos `Bloco(b)` do domínio da acção `MoveParaMesa` e obteremos acções em que se move a `Mesa` de um bloco para a `Mesa` entre outras acções incorrectas. Vejam que embora seja gerada a partir do esquema, essa acção nunca poderia ser realizada porque nunca veria satisfeita uma das suas pré-condições: `Livre(Mesa)`.

In [40]:
acoes = [Action('MoveParaMesa(b, x)',
                precond='Sobre(b, x) & Livre(b)',
                effect='Sobre(b, Table) & Livre(x) & ~Sobre(b, x)',
                domain='Bloco(x)'),
         Action('Move(b, x, y)',
                precond='Sobre(b, x) & Livre(b) & Livre(y)',
                effect='Sobre(b, y) & Livre(x) & ~Sobre(b, x) & ~Livre(y)',
                domain='Bloco(b) & Bloco(y)')]
print(acoes)

threeBlockTower = PlanningProblem(initial=kb, goals=goal, actions=acoes, domain=dominio)

threeBlockTower.expand_actions()

[MoveParaMesa(b, x), Move(b, x, y)]


[MoveParaMesa(B, A),
 MoveParaMesa(C, B),
 MoveParaMesa(C, A),
 MoveParaMesa(A, B),
 MoveParaMesa(Mesa, B),
 MoveParaMesa(Mesa, A),
 Move(B, C, A),
 Move(B, Mesa, A),
 Move(A, C, B),
 Move(A, Mesa, B)]

Verifiquemos o que acontece se os dominios nos esquemas das acções não forem definidos, i.e., a definição de bloco for integrada nas pré-condições dos esquemas de ações. Vamos definir agora o problema *TorreBlocos* onde o esquema de ações não tem o atributo domínio explicitamente definido.

In [41]:
# ações
ac = [Action('MoveParaMesa(b, x)',
                precond='Sobre(b, x) & Livre(b) & Bloco(b) & Bloco(x)',
                effect='Sobre(b, Mesa) & Livre(x) & ~Sobre(b, x)'
               ),
         Action('Move(b, x, y)',
                precond='Sobre(b, x) & Livre(b) & Livre(y) & Bloco(b) & Bloco(y)',
                effect='Sobre(b, y) & Livre(x) & ~Sobre(b, x) & ~Livre(y)'
               )]

torreBlocos = PlanningProblem(initial=kb, goals=goal, actions=ac, domain=dominio)

In [42]:
torreBlocos.expand_actions()

[MoveParaMesa(B, C),
 MoveParaMesa(B, A),
 MoveParaMesa(B, Mesa),
 MoveParaMesa(C, B),
 MoveParaMesa(C, A),
 MoveParaMesa(C, Mesa),
 MoveParaMesa(A, B),
 MoveParaMesa(A, C),
 MoveParaMesa(A, Mesa),
 MoveParaMesa(Mesa, B),
 MoveParaMesa(Mesa, C),
 MoveParaMesa(Mesa, A),
 Move(B, C, A),
 Move(B, C, Mesa),
 Move(B, A, C),
 Move(B, A, Mesa),
 Move(B, Mesa, C),
 Move(B, Mesa, A),
 Move(C, B, A),
 Move(C, B, Mesa),
 Move(C, A, B),
 Move(C, A, Mesa),
 Move(C, Mesa, B),
 Move(C, Mesa, A),
 Move(A, B, C),
 Move(A, B, Mesa),
 Move(A, C, B),
 Move(A, C, Mesa),
 Move(A, Mesa, B),
 Move(A, Mesa, C),
 Move(Mesa, B, C),
 Move(Mesa, B, A),
 Move(Mesa, C, B),
 Move(Mesa, C, A),
 Move(Mesa, A, B),
 Move(Mesa, A, C)]

Reparem que foram instanciadas 36 ações (o dobro das ações instanciadas no problema *torre3blocos*). Existem acções como por exemplo `MoveParaMesa(B, Mesa)` (i.e., mover o bloco B que está em cima da mesa para a mesa) a qual não faz nenhum sentido, sendo uma acção totalmente redundante.

Muitas destas ações não serão utilizadas porque as pré-condições irão falhar. No entanto, o planeador irá testá-las para saber quais poderá ou não aplicar a um estado, o que fará com que a procura possa ser demorada.

### Aplicação das acções e verificação de satisfação dos objectivos

Redefinamos então o problema dos blocos correctamente.

In [43]:
# conhecimento base
kb = 'Sobre(A, Mesa) & Sobre(B, Mesa) & Sobre(C, A) & Livre(B) & Livre(C)'
# dominio
dominio = 'Bloco(A) & Bloco(B) & Bloco(C)'
# objetivo
goal = 'Sobre(B, C) & Sobre(A, B)'

torre3blocos = PlanningProblem(initial=kb, goals=goal, actions=acoes, domain=dominio)

Vamos aplicar a ação `MoveParaMesa(C,A)` que corresponde ao esquema de ação `MoveParaMesa(x,y)`, quando `x=C` e `y=A`, e verificar se atingiu o objetivo. Reparem que ao aplicarmos uma ação, através do método `act` o estado mudou, sendo o estado resultante guardado no atributo `initial` do problema.

In [44]:
# conhecimento base
print('Conhecimento base:', torre3blocos.initial)
# vamos aplicar uma ação ao estado inicial do problema
torre3blocos.act(expr('MoveParaMesa(C,A)'))
print('Estado seguinte:', torre3blocos.initial)
print('Objectivo atingido?',torre3blocos.goal_test())

Conhecimento base: [Bloco(A), Bloco(B), Bloco(C), Livre(B), Livre(C), Sobre(A, Mesa), Sobre(B, Mesa), Sobre(C, A)]
Estado seguinte: [Bloco(A), Bloco(B), Bloco(C), Livre(A), Livre(B), Livre(C), NotSobre(C, A), Sobre(A, Mesa), Sobre(B, Mesa), Sobre(C, Table)]
Objectivo atingido? False


Ao movermos o bloco C, que está em cima do A, para a mesa, desapareceu do estado a expressão `Sobre(C,A)` e foi adicionado `NotSobre(C,A)`. Foram ainda adicionadas mais algumas expressões para descreverem o estado atual: 
* `Livre(A)` - o bloco A deixou de ter o bloco C em cima, i.e., passou a estar livre
* `Sobre(C, Mesa)` - o bloco C passou a estar em cima da mesa

Vamos aplicar agora a ação `Move(C, Mesa, A)` a `initial` e ver o que acontece.

In [45]:
print('Estado atual:', torre3blocos.initial)
torre3blocos.act(expr('Move(C,Table,A)'))
print('Estado seguinte:', torre3blocos.initial)
print('Objectivo atingido?',torre3blocos.goal_test())

Estado atual: [Bloco(A), Bloco(B), Bloco(C), Livre(A), Livre(B), Livre(C), NotSobre(C, A), Sobre(A, Mesa), Sobre(B, Mesa), Sobre(C, Table)]
Estado seguinte: [Bloco(A), Bloco(B), Bloco(C), Livre(B), Livre(C), Livre(Table), NotLivre(A), NotSobre(C, Table), Sobre(A, Mesa), Sobre(B, Mesa), Sobre(C, A)]
Objectivo atingido? False


Reparem agora que voltamos a colocar o bloco C em cima do A, desaparecendo a expressão `NotSobre(C,A)` e `Sobre(C, Table)` e sendo substituidas por `Sobre(C,A)` e `NotSobre(C,Table)`, respetivamente.

### Exercício 4
Modifiquem o objetivo do problema dos blocos, considerando que apenas queremos que o bloco B passe a ter um outro bloco por cima. Apliquem ao estado inicial do problema a acção de mover o C para cima do B e confirmem se atingiu o objectivo.

In [46]:
# Solução do exercício 4

torreBlocos = PlanningProblem(initial=kb, goals='NotLivre(B)', actions=acoes, domain=dominio)
print(torreBlocos.initial)
torreBlocos.act(expr('Move(C,A,B)'))
print(torreBlocos.initial)
print('Atingiu o objetivo?', torreBlocos.goal_test())

[Bloco(A), Bloco(B), Bloco(C), Livre(B), Livre(C), Sobre(A, Mesa), Sobre(B, Mesa), Sobre(C, A)]
[Bloco(A), Bloco(B), Bloco(C), Livre(A), Livre(C), NotLivre(B), NotSobre(C, A), Sobre(A, Mesa), Sobre(B, Mesa), Sobre(C, B)]
Atingiu o objetivo? True


Regressemos ao problema inicial:

In [47]:
# conhecimento base
kb = 'Sobre(A, Mesa) & Sobre(B, Mesa) & Sobre(C, A) & Livre(B) & Livre(C)'
# dominio
dominio = 'Bloco(A) & Bloco(B) & Bloco(C)'
# objetivo
goal = 'Sobre(B, C) & Sobre(A, B)'

acoes = [Action('MoveParaMesa(b, x)',
                precond='Sobre(b, x) & Livre(b)',
                effect='Sobre(b, Mesa) & Livre(x) & ~Sobre(b, x)',
                domain='Bloco(b) & Bloco(x)'),
         Action('Move(b, x, y)',
                precond='Sobre(b, x) & Livre(b) & Livre(y)',
                effect='Sobre(b, y) & Livre(x) & ~Sobre(b, x) & ~Livre(y)',
                domain='Bloco(b) & Bloco(y)')]

torre3blocos = PlanningProblem(initial=kb, goals=goal, actions=acoes, domain=dominio)

e vamos agora aplicar uma série de ações e verificar se atingimos o objetivo.

In [48]:
solution = [expr('MoveParaMesa(C,A)'),
            expr('Move(B, Mesa, C)'),
            expr('Move(A, Mesa, B)')
           ]

for action in solution:
    torre3blocos.act(action)

print('Atingiu o goal?', torre3blocos.goal_test())

Atingiu o goal? True


Vejemos o estado final

In [49]:
print('Estado atual:', torre3blocos.initial)

Estado atual: [Bloco(A), Bloco(B), Bloco(C), Livre(A), Livre(Mesa), NotLivre(B), NotLivre(C), NotSobre(A, Mesa), NotSobre(B, Mesa), NotSobre(C, A), Sobre(A, B), Sobre(B, C), Sobre(C, Mesa)]


### Um Problema da Formulação:
Notem que o predicado `Livre(Mesa)` aparece no estado e não faz sentido. Dissemos logo na descrição do problema que a Mesa é infinita, i.e. está sempre livre.

A acção `Move(x,y,z)` deveria ser apenas entre blocos de modo a que fizesse sentido o bloco y ficar livre, i.e., y não deveria ser a mesa (Mesa).

Deveriamos ter um outro esquema de acção específico para mover um bloco que estivesse na mesa para cima de outro bloco.

### Exercício 5
Formulem o problema de modo a resolver o problema da formulação indicado em cima, resultando em que não apareçam os predicados `Livre(Table)`. Apliquem as acções que levem ao estado final e confirmem que essas expressões não aparecem.

In [50]:
# Solução do 5

# cohecimento base
kb = 'Sobre(A, Mesa) & Sobre(B, Mesa) & Sobre(C, A) & Livre(B) & Livre(C)'
# dominio
dominio = 'Bloco(A) & Bloco(B) & Bloco(C)'
# objetivo
goal = 'Sobre(B, C) & Sobre(A, B)'

# ações
acoes = [Action('MoveParaMesa(b, x)',
                precond='Sobre(b, x) & Livre(b) & Bloco(b) & Bloco(x)',
                effect='Sobre(b, Mesa) & Livre(x) & ~Sobre(b, x)',
                domain='Bloco(b) & Bloco(x)'
               ),
         Action('Move(b, x, y)',
                precond='Sobre(b, x) & Livre(b) & Livre(y) & Bloco(b) & Bloco(y)',
                effect='Sobre(b, y) & Livre(x) & ~Sobre(b, x) & ~Livre(y)',
                domain='Bloco(b) & Bloco(x) & Bloco(y)'
               ),
         Action('MoveDaMesa(b, y)',
                precond='Sobre(b, Mesa) & Livre(b) & Livre(y)',
                effect='Sobre(b, y)  & ~Sobre(b, Mesa) & ~Livre(y)',
                domain='Bloco(b) & Bloco(y)'
               )]

torreBlocos = PlanningProblem(initial=kb, goals=goal, actions=acoes, domain=dominio)

solution = [expr('MoveParaMesa(C,A)'),
               expr('MoveDaMesa(B, C)'),
               expr('MoveDaMesa(A, B)')
           ]

for action in solution:
    torreBlocos.act(action)

print('Estado atual:\n', torreBlocos.initial)

print('\nAtingiu o objectivo?', torreBlocos.goal_test())

Estado atual:
 [Bloco(A), Bloco(B), Bloco(C), Livre(A), NotLivre(B), NotLivre(C), NotSobre(A, Mesa), NotSobre(B, Mesa), NotSobre(C, A), Sobre(A, B), Sobre(B, C), Sobre(C, Mesa)]

Atingiu o objectivo? True


### Outro Problema da Formulação:
Imaginem que queremos ter como objecctivo que o bloco C não esteja sobre a mesa, o que quer dizer que se estiver sobre A ou sobre B o objectivo estará satisfeito. No entanto com o podemos dizer? Podemos usar a expressão `~Sobre(C,Mesa)`. Se aplicarmos o `goal_test` ao estado inicial do problema na imagem, obteremos `False` e deveria ser `True`.

Regressemos ao problema inicial, mas em que redefinimos o objectivo:

In [51]:
# conhecimento base
kb = 'Sobre(A, Mesa) & Sobre(B, Mesa) & Sobre(C, A) & Livre(B) & Livre(C)'
# dominio
dominio = 'Bloco(A) & Bloco(B) & Bloco(C)'
# objetivo
goal = '~Sobre(C, Mesa)'

torre3blocos = PlanningProblem(initial=kb, goals=goal, actions=acoes, domain=dominio)
torre3blocos.initial

[Bloco(A),
 Bloco(B),
 Bloco(C),
 Livre(B),
 Livre(C),
 Sobre(A, Mesa),
 Sobre(B, Mesa),
 Sobre(C, A)]

Verifiquemos que não satisfaz o objectivo.

In [52]:
torre3blocos.goal_test()

False

### Exercício 6
Notem que não há conhecimento que indique que se um bloco está aqui isso quer dizer que não está acolá. Temos de o dizer explicitamente. Modifiquem o modelo de modo a que o conhecimento de base seja enriquecido com as negações.

In [54]:
# situação inicial
kb = 'Sobre(A, Mesa) & Sobre(B, Mesa) & Sobre(C, A) & Livre(B) & Livre(C) & ~Livre(A) & ~Sobre(A,B) & ~Sobre(A,C) &\
      ~Sobre(B,A) & ~Sobre(B,C) & ~Sobre(C,B) & ~Sobre(C,Mesa)'
# dominio
dominio = 'Bloco(A) & Bloco(B) & Bloco(C)'
# objetivo
goal = '~Sobre(C, Mesa)'
# ações
# ações
acoes = [Action('MoveParaMesa(b, x)',
                precond='Sobre(b, x) & Livre(b) & Bloco(b) & Bloco(x)',
                effect='Sobre(b, Mesa) & Livre(x) & ~Sobre(b, x)',
                domain='Bloco(b) & Bloco(x)'
               ),
         Action('Move(b, x, y)',
                precond='Sobre(b, x) & Livre(b) & Livre(y) & Bloco(b) & Bloco(y)',
                effect='Sobre(b, y) & Livre(x) & ~Sobre(b, x) & ~Livre(y)',
                domain='Bloco(b) & Bloco(x) & Bloco(y)'
               ),
         Action('MoveDaMesa(b, y)',
                precond='Sobre(b, Mesa) & Livre(b) & Livre(y)',
                effect='Sobre(b, y)  & ~Sobre(b, Mesa) & ~Livre(y)',
                domain='Bloco(b) & Bloco(y)'
               )]

torre3blocos = PlanningProblem(initial=kb, goals=goal, actions=acoes, domain=dominio)
torre3blocos.initial
torre3blocos.goal_test()

True

## Outro Exemplo: Viagem à Roménia

Vamos tentar planear uma viagem  considerando um mapa simplificado da Roménia. 
<img src="romania.png" width="250">

O conhecimento base indica apenas a nossa localização no mapa (*Sibiu*). A informação sobre que cidades estão interligadas por estradas e quais têm aeroportos é informação estática do nosso problema, sendo definida no dominio. No entanto, não é necessário representar cidades, como por exemplo, `Cidade(Sibiu)`, porque todas são cidades. Para além disso, não iremos representar ligações simétricas entre cidades.

In [55]:
# conhecimento base
kb = 'Em(Sibiu)'
# dominio
prob_dominio = 'Aeroporto(Sibiu) & Aeroporto(Buchuresti) & Aeroporto(Craiova) & Ligada(Buchuresti,Pitesti)' + \
                ' & Ligada(Pitesti,Rimnicu) & Ligada(Rimnicu,Sibiu) & Ligada(Sibiu,Fagaras)' + \
                '& Ligada(Fagaras,Buchuresti) & Ligada(Pitesti,Craiova) & Ligada(Craiova,Rimnicu)'

Vamos agora descrever esquemas de ações para o problema. Sabemos que podemos ir de carro entre quaisquer lugares ligados. Mas como é evidente, também podemos ir de avião entre Sibiu, Buchuresti e Craiova.

Vamos definir as ações de voar desta forma:

In [56]:
# Voa de qualquer cidade com aeroporto para qualquer outra com aeroporto
precond = 'Em(x)'
efeito = 'Em(y) & ~Em(x)'
dominio = 'Aeroporto(x) & Aeroporto(y)'
voa = Action('Voa(x, y)', precond, efeito, dominio)

E qualquer ação de conduzir desta forma:

In [57]:
# vai por estrada de x para y
precond = 'Em(x)'
efeito = 'Em(y) & ~Em(x)'
dominio = 'Ligada(x,y)'
conduz = Action('Conduz(x, y)', precond, efeito, dominio)

# vai por estrada de y para x -- usa a ligação simétrica Ligada(y,x)
precond = 'Em(x)'
efeito = 'Em(y) & ~Em(x)'
dominio = 'Ligada(y,x)'
conduz_sim = Action('ConduzInv(x, y)', precond, efeito, dominio)

O nosso objetivo é definido como chegar a Buchuresti.

In [58]:
objectivos = 'Em(Buchuresti)'

Assim, podemos definir o seguinte problema de planeamento:

In [59]:
prob = PlanningProblem(kb, objectivos, [voa, conduz, conduz_sim], prob_dominio)

Tal como anteriormente vamos testar se atingiu o objetivo e aplicar algumas ações a este problema.

In [60]:
print('Objetivo?', prob.goal_test())

Objetivo? False


In [61]:
print('Conhecimento base:', prob.initial)

Conhecimento base: [Aeroporto(Buchuresti), Aeroporto(Craiova), Aeroporto(Sibiu), Em(Sibiu), Ligada(Buchuresti, Pitesti), Ligada(Craiova, Rimnicu), Ligada(Fagaras, Buchuresti), Ligada(Pitesti, Craiova), Ligada(Pitesti, Rimnicu), Ligada(Rimnicu, Sibiu), Ligada(Sibiu, Fagaras)]


Reparem que o dominio do problema é sempre adicionado ao conhecimento base.

Vamos agora aplicar uma ação de voar entre Sibiu e Bucharest.

In [62]:
prob.act(expr('Voa(Sibiu, Buchuresti)'))
print('Estado após aplicar ação de voar:', prob.initial)

Estado após aplicar ação de voar: [Aeroporto(Buchuresti), Aeroporto(Craiova), Aeroporto(Sibiu), Em(Buchuresti), Ligada(Buchuresti, Pitesti), Ligada(Craiova, Rimnicu), Ligada(Fagaras, Buchuresti), Ligada(Pitesti, Craiova), Ligada(Pitesti, Rimnicu), Ligada(Rimnicu, Sibiu), Ligada(Sibiu, Fagaras), NotEm(Sibiu)]


Vamos agora assumir que estamos em Pitesti e queremos viajar para Buchuresti. Reparem que não existem voos entre estas duas cidades, o que significa que terá de ser uma viagem por carro.

In [63]:
prob = PlanningProblem('Em(Pitesti)', objectivos, [voa, conduz, conduz_sim], prob_dominio)
print('Conhecimento base:', prob.initial)

Conhecimento base: [Aeroporto(Buchuresti), Aeroporto(Craiova), Aeroporto(Sibiu), Em(Pitesti), Ligada(Buchuresti, Pitesti), Ligada(Craiova, Rimnicu), Ligada(Fagaras, Buchuresti), Ligada(Pitesti, Craiova), Ligada(Pitesti, Rimnicu), Ligada(Rimnicu, Sibiu), Ligada(Sibiu, Fagaras)]


Vamos fazer a viagem por carro entre Pitesti e Buchuresti. Reparem que na nossa *knowledge_base* definimos a ligação `Ligada(Buchuresti, Pitesti)`, mas não ao contrário. O que significa que não poderemos aplicar a acção `Conduz`iremos usar a segunda ação `ConduzInv(Pitesti, Buchuresti)`.

In [64]:
prob.act(expr('ConduzInv(Pitesti,Buchuresti)'))
print('Estado após aplicar ação de conduzir:', prob.initial)

Estado após aplicar ação de conduzir: [Aeroporto(Buchuresti), Aeroporto(Craiova), Aeroporto(Sibiu), Em(Buchuresti), Ligada(Buchuresti, Pitesti), Ligada(Craiova, Rimnicu), Ligada(Fagaras, Buchuresti), Ligada(Pitesti, Craiova), Ligada(Pitesti, Rimnicu), Ligada(Rimnicu, Sibiu), Ligada(Sibiu, Fagaras), NotEm(Pitesti)]


In [65]:
print('Objetivo?',prob.goal_test())

Objetivo? True


## Mais um exemplo: Comer bolo

Vamos formular um problema de comer um bolo, i.e., o agente quer comer um bolo (este é objetivo). O que significa que se o agente tem um bolo, come-o. Se não tem um bolo, faz um, e ao fazer um bolo, passa a ter um e, consequentemente, pode comê-lo. Vamos assumir que inicialmente não existe bolo. 

Este é um exemplo muito simples onde se pretende mostrar que é possivel formular problemas de planeamento cujo o conhecimento base e as pré-condições têm expressões negadas. 

In [66]:
inicial = '~Ha(Bolo)'
objectivo = 'Comeu(Bolo)'

As ações serão comer um bolo `Come(Bolo)` e fazer um bolo `Cozinha(Bolo)`. Aqui temos que ter atenção às pré-condições que iremos definir. Só poderemos fazer um bolo se não existir um bolo.

In [67]:
accoes = [Action('Come(Bolo)',
                  precond='Ha(Bolo)',
                  effect='Comeu(Bolo) & ~Ha(Bolo)'),
          Action('Cozinha(Bolo)',
                  precond='~Ha(Bolo)',
                  effect='Ha(Bolo)')]

Criemos o problema de planeamento e espreitemos o seu estado inicial

In [68]:
bolo = PlanningProblem(initial=inicial, goals=objectivo, actions=accoes)

In [69]:
bolo.initial

[NotHa(Bolo)]

A ordem pela qual aplicamos as ações é importante, pois não podemos aplicar a ação de comer um bolo se esse bolo não existir, ou seja, tem de se fazer o bolo para que a pré-condição da ação `Come(Bolo)` seja satisfeita.

In [70]:
bolo.act(expr('Come(Bolo)'))

Exception: Action 'Come(Bolo)' pre-conditions not satisfied

Vamos aplicar a ação de fazer um bolo e depois comer o bolo, para verificarmos o objetivo.

In [71]:
print('Estado inicial:', bolo.initial)
bolo.act(expr('Cozinha(Bolo)'))
print('Estado após ação de fazer bolo:', bolo.initial)

Estado inicial: [NotHa(Bolo)]
Estado após ação de fazer bolo: [Ha(Bolo)]


Reparem que o estado atual consiste em ter o bolo, logo pode ser aplicada a ação de comer o bolo, a pré-condição está satisfeita.

In [72]:
bolo.act(expr('Come(Bolo)'))
print('Estado após ação comer bolo:', bolo.initial)
print('Atingiu objetivo?', bolo.goal_test())

Estado após ação comer bolo: [Comeu(Bolo), NotHa(Bolo)]
Atingiu objetivo? True


## Exercício 7

Consideremos o problema de trocar um pneu furado num carro. O pneu furado (`Furado`) encontra-se no eixo do carro (`Eixo`) e o pneu sobressalente (`Sobressalente`) na bagageira (`Bagageira`). O objetivo é colocar o pneu sobressalente no eixo do carro e o pneu furado na bagageira mas é preciso usar o chão (`Chao`) como espaço intermédio. Formule este problema como um problema de planeamento.

**Dica:** Temos como dominio do problema os dois pneus (`Pneu(Furado) & Pneu(Sobressalente)`).  Por exemplo, podemos modelizar com uma única acção: `Coloca` para colocar um objecto que está num determinado lugar noutro (por exemplo, retira o pneu do chão e colocá-lo no eixo).

In [73]:
# Resolução do exercicio 6

pneu_furado = PlanningProblem(initial='Esta(Furado, Eixo) & Esta(Sobressalente, Bagageira)',
                             goals='Esta(Sobressalente, Eixo) & Esta(Furado, Bagageira)',
                             actions=[
                                 Action('Coloca(obj, loc,novoloc)',
                                       precond='Esta(obj, loc)',
                                       effect='Esta(obj, novoloc) & ~Esta(obj, loc)',
                                       domain='Pneu(obj)'),
                             ],
                             domain='Pneu(Sobressalente) & Pneu(Furado)'                             
                            )

# Aplicar algumas ações
print('Vamos colocar o pneu sobressalente no chão')
pneu_furado.act(expr('Coloca(Sobressalente, Bagageira, Chao)'))
print(pneu_furado.initial)
print('Vamos colocar o pneu furado também no chão')
pneu_furado.act(expr('Coloca(Furado, Eixo, Chao)'))
print(pneu_furado.initial)
print('Vamos colocar o pneu sobressalente no eixo')
pneu_furado.act(expr('Coloca(Sobressalente,Chao,Eixo)'))
print(pneu_furado.initial)
print('Vamos colocar o pneu furado na bagageira')
pneu_furado.act(expr('Coloca(Furado,Chao,Bagageira)'))
print(pneu_furado.initial)
print('Objetivo?', pneu_furado.goal_test())

Vamos colocar o pneu sobressalente no chão
[Esta(Furado, Eixo), Esta(Sobressalente, Chao), NotEsta(Sobressalente, Bagageira), Pneu(Furado), Pneu(Sobressalente)]
Vamos colocar o pneu furado também no chão
[Esta(Furado, Chao), Esta(Sobressalente, Chao), NotEsta(Furado, Eixo), NotEsta(Sobressalente, Bagageira), Pneu(Furado), Pneu(Sobressalente)]
Vamos colocar o pneu sobressalente no eixo
[Esta(Furado, Chao), Esta(Sobressalente, Eixo), NotEsta(Furado, Eixo), NotEsta(Sobressalente, Bagageira), NotEsta(Sobressalente, Chao), Pneu(Furado), Pneu(Sobressalente)]
Vamos colocar o pneu furado na bagageira
[Esta(Furado, Bagageira), Esta(Sobressalente, Eixo), NotEsta(Furado, Chao), NotEsta(Furado, Eixo), NotEsta(Sobressalente, Bagageira), NotEsta(Sobressalente, Chao), Pneu(Furado), Pneu(Sobressalente)]
Objetivo? True


## Planeamento como procura em espaços de estados

A descrição de um problema de planeamento define um problema de procura: podemos procurar a partir do estado inicial através de espaços de estados, movimento para a frente ('forward') procurando o objetivo. Uma das vantagens da representação declarativa dos esquemas de ações é que também se pode procurar "backward" a partir do objetivo, procurando o estado inicial.

As classes `ForwardPlan` e `BackwardPlan` são duas subclasses da classe `Problem`, que como se recordam é usada na formulação de problemas de espaços de estados. Assim, ambas as subclasses têm definidos métodos já conhecidos, nomeadamente:
* `actions(self, state)` - este método, dado um estado, devolve a lista de todas as ações possiveis nesse estado;
* `result(self, state, action)` - dados um estado e uma ação, este método devolve o estado resultante da execução da ação **action**, no estado **state**
* `goal_test(self, state)` - este método retornará `True` nos casos em que o estado fornecido (**state**) seja o estado final ou membro dos estados finais

Nesta aula iremos apenas trabalhar com a classe `ForwardPlan`.

### Espaços de estados: classe `ForwardPlan`

A classe `ForwardPlan` é uma sub-classe da classe `Problem`, isto significa que é necessário definir o goal e o estado inicial no construtor. No *ForwardPlan* o estado inicial corresponde ao conhecimento base do problema de planeamento e o goal aos objetivos. Vejamos brevemente o construtor desta classe:
```python
def __init__(self, planning_problem, verbose=False):
    super().__init__(associate('&', planning_problem.initial), associate('&', planning_problem.goals))
    self.planning_problem = planning_problem
    self.expanded_actions = self.planning_problem.expand_actions()
    if verbose:
        print(self.expanded_actions)
```
Reparem que a instrução `super().__init__` está a definir os atributos da classe `Problem` que esta sub-classe vai herdar, é aqui que definimos o estado inicial como sendo o conhecimento base (`planning_problem.initial`) e o goal como sendo o objetivo do problema de planeamento (`planning_problem.goals`). Para além disso, estamos a instanciar todas as ações viáveis (no atributo `expanded_actions`) a partir dos esquemas de ações definidos no problema de planeamento.

Recordemos o problema da torre dos três blocos definido anteriormente. Já definimos os estado inicial, objetivos, e ações de uma forma abstrata, recorrendo à classe `PlanningProblem`.
<img src="3-blocks.png" width="300">

In [74]:
# situação inicial
kb = 'Sobre(A, Mesa) & Sobre(B, Mesa) & Sobre(C, A) & Livre(B) & Livre(C) & ~Livre(A) & ~Sobre(A,B) & ~Sobre(A,C) &\
      ~Sobre(B,A) & ~Sobre(B,C) & ~Sobre(C,B) & ~Sobre(C,Mesa)'
# dominio
dominio = 'Bloco(A) & Bloco(B) & Bloco(C)'
# objetivo
goal = 'Sobre(B, C) & Sobre(A, B)'
# ações
# ações
acoes = [Action('MoveParaMesa(b, x)',
                precond='Sobre(b, x) & Livre(b) & Bloco(b) & Bloco(x)',
                effect='Sobre(b, Mesa) & Livre(x) & ~Sobre(b, x)',
                domain='Bloco(b) & Bloco(x)'
               ),
         Action('Move(b, x, y)',
                precond='Sobre(b, x) & Livre(b) & Livre(y) & Bloco(b) & Bloco(y)',
                effect='Sobre(b, y) & Livre(x) & ~Sobre(b, x) & ~Livre(y)',
                domain='Bloco(b) & Bloco(x) & Bloco(y)'
               ),
         Action('MoveDaMesa(b, y)',
                precond='Sobre(b, Mesa) & Livre(b) & Livre(y)',
                effect='Sobre(b, y)  & ~Sobre(b, Mesa) & ~Livre(y)',
                domain='Bloco(b) & Bloco(y)'
               )]

torre3blocos = PlanningProblem(initial=kb, goals=goal, actions=acoes, domain=dominio)

Agora iremos transformar este problema de planeamento num problema de espaços de estados, em que queremos começar com o conhecimento base e ir aplicando as acções (para a frente) até atingir um estado que satisfaça os objectivos.

Vamos criar uma instância da classe `ForwardPlan`, passando-lhe a instância de `PlanningProblem`:

In [75]:
p = ForwardPlan(torre3blocos)

Ao instanciar um problema como espaços de estados, todas as ações viáveis serão geradas a partir dos esquemas de ações abstratos definidos no problema de planeamento.

Para verificarmos as ações geradas, temos de ativar o modo display.

In [76]:
p = ForwardPlan(torre3blocos, display=True)

As 18 Ações expandidas: [MoveParaMesa(B, C), MoveParaMesa(B, A), MoveParaMesa(C, B), MoveParaMesa(C, A), MoveParaMesa(A, B), MoveParaMesa(A, C), Move(B, C, A), Move(B, A, C), Move(C, B, A), Move(C, A, B), Move(A, B, C), Move(A, C, B), MoveDaMesa(B, C), MoveDaMesa(B, A), MoveDaMesa(C, B), MoveDaMesa(C, A), MoveDaMesa(A, B), MoveDaMesa(A, C)]


Reparem que foram geradas todas as ações viáveis entre os 3 blocos e entre blocos e mesa. Além disso, nenhuma das ações  dos esquemas `MoveParaMesa(Mesa, x)`, `MoveParaMesa(x, Mesa)`, `Move(Mesa, x, y)` e `Move(x, y, Mesa)`, em que `x` e `y` são Mesa foi gerada, i.e. podem ser A, B, C, foi gerada. Este tipo de ações não foi gerado por causa do atributo dominio que existe no objeto `Action`. 

Existem quatro objetos definidos: A, B, C e Mesa, e todas as permutações destes objetos irão ser testadas em cada um dos esquemas de ações. No entanto, apenas algumas ações serão válidas por causa da filtragem do dominio (tal como explicado anteriormente), que só aceita alguns objetos como sendo blocos.

O estado inicial é uma conjunção, uma `Exprs`, e corresponde ao conhecimento de base da instância do problema de planeamento que é passado como input no construtor.

Vamos verificar o estado inicial e quais as ações possíveis para o estado inicial.

In [80]:
p.initial

(Bloco(A) & Bloco(B) & Bloco(C) & Livre(B) & Livre(C) & NotLivre(A) & NotSobre(A, B) & NotSobre(A, C) & NotSobre(B, A) & NotSobre(B, C) & NotSobre(C, B) & NotSobre(C, Mesa) & Sobre(A, Mesa) & Sobre(B, Mesa) & Sobre(C, A))

Para saber quais as acções possíveis, usamos o método standard `actions`:

In [81]:
a = p.actions(p.initial)
print(a)

[MoveParaMesa(C, A), Move(C, A, B), MoveDaMesa(B, C)]


Vamos mover o bloco C que está em cima de A para a Mesa e verificar o estado resultante, usando a função `result`:

In [82]:
e1 = p.result(p.initial,a[0])
print(e1)

(Bloco(A) & Bloco(B) & Bloco(C) & Livre(A) & Livre(B) & Livre(C) & NotSobre(A, B) & NotSobre(A, C) & NotSobre(B, A) & NotSobre(B, C) & NotSobre(C, A) & NotSobre(C, B) & Sobre(A, Mesa) & Sobre(B, Mesa) & Sobre(C, Mesa))


E quais as acções agora?

In [83]:
accoes=p.actions(e1)
print(accoes)

[MoveDaMesa(B, C), MoveDaMesa(B, A), MoveDaMesa(C, B), MoveDaMesa(C, A), MoveDaMesa(A, B), MoveDaMesa(A, C)]


Vamos voltar a mover o bloco C que está em cima da mesa de volta para A e verificar o estado resultante, usando a função `result`:

In [84]:
e2 = p.result(e1,accoes[3])
print(e2)

(Bloco(A) & Bloco(B) & Bloco(C) & Livre(B) & Livre(C) & NotLivre(A) & NotSobre(A, B) & NotSobre(A, C) & NotSobre(B, A) & NotSobre(B, C) & NotSobre(C, B) & NotSobre(C, Mesa) & Sobre(A, Mesa) & Sobre(B, Mesa) & Sobre(C, A))


Se compararmos o estado inicial com este que decorre da execução de uma acção e da sua inversa:

In [85]:
print(p.initial)
e2==p.initial

(Bloco(A) & Bloco(B) & Bloco(C) & Livre(B) & Livre(C) & NotLivre(A) & NotSobre(A, B) & NotSobre(A, C) & NotSobre(B, A) & NotSobre(B, C) & NotSobre(C, B) & NotSobre(C, Mesa) & Sobre(A, Mesa) & Sobre(B, Mesa) & Sobre(C, A))


True

##### Procura em espaços de estados

A procura `Forward` através dos espaços de estados, começa no estado inicial e usa as ações do problema para procurar "para a frente" até atingir um estado que satisfaça a expressão indicada no atributo `goal`.

Vamos, por exemplo, aplicar a procura em largura para encontrar soluções para o nosso problema dos blocos `p`, problema definido anteriormente onde cada acção tem o domínio bem definido (são geradas 18 ações viáveis para serem usadas durante a procura, embora apenas um subconjunto delas é que pode satisfazer as pré-condições).

Usemos a função **breadth_first_graph_search_plus()**, que faz uma procura em largura em grafo e em que o objectivo é testado quando um estado é visitado, i.e., quando se geram os sucessores.

In [86]:
solucao_mundo_blocos = breadth_first_graph_search_plus(p).solution()
solucao_mundo_blocos = list(map(lambda action: Expr(action.name, *action.args), solucao_mundo_blocos))
print('Solução:',solucao_mundo_blocos)

Solução: [MoveParaMesa(C, A), MoveDaMesa(B, C), MoveDaMesa(A, B)]


## Exercício 8

Temos duas cargas em dois aeroportos diferentes: carga 1 em Lisboa (LX) e carga 2 no Porto (OPO). O nosso objetivo é enviar cada carga para o outro aeroporto. Temos dois aviões para nos ajudar a completar essa tarefa. 

a) Formule este problema como um problema de planeamento.

**Dicas:**  
1. O dominio indica os vários objetos e os respetivos tipos, i.e., `Carga(C1)` e `Aviao(P1)` e `Aeroporto(LX)` e ...
2. O objetivo será a carga C1 estar no Porto (OPO), `Em(C1,OPO)` e a carga C2 estar em Lisboa (LX),  `Em(C2,LX)`
3. O problema pode ser definido com três ações: 
    * `Carrega(c,p,a)` - carregar carga `c` no avião `p` que está no aeroport `a`, ie, `c` e `p`têm de estar ambos no aeroporto `a`. Esta ação terá o efeito de a carga estar dentro do avião (`Dentro(c,p)`) e deixar de estar no aeroporto (`~Em(c,a)`).
    * `Descarrega(c,p,a)` - descarregar carga `c` do avião `p` que está no aeroporto `a`. Esta ação terá o efeito de a carga estar no aeroporto (`Em(c,a)`) e não estar dentro do avião (`~Dentro(c,p)`).
    * `Voa(p,f,to)` - voo do avião `p` que parte de `f` (não se pode usar *from*, o planeador não aceita esse literal) com destino a `to`
4. Depois de definirem o problema podem aplicar o seguinte conjunto de ações que corresponde à solução do problema:
```python
solution = [expr("Carrega(C1 , P1, LX)"),
            expr("Voa(P1, LX, OPO)"),
            expr("Descarrega(C1, P1, OPO)"),
            expr("Carrega(C2, P2, OPO)"),
            expr("Voa(P2, OPO, LX)"),
            expr("Desccarrega(C2, P2, LX)")]
```

b) Obtenha os tempos de execução e os paths (activando o modo display na função *breadth_first_graph_search_plus*, `display=True`) no Forward.

In [105]:
# Resolução do exercicio 7

air_Cargo = PlanningProblem(initial='Em(C1,LX) & Em(C2,OPO) & Em(P1,LX) & Em(P2,OPO)',
                     goals='Em(C1,OPO) & Em(C2,LX)',
                     actions=[
                         Action('Carrega(c, p, a)',
                               precond='Em(c, a) & Em(p, a)',
                               effect='Dentro(c, p) & ~Em(c, a)',
                               domain='Carga(c) & Aviao(p) & Aeroporto(a)'),
                         Action('Descarrega(c, p, a)',
                               precond='Dentro(c, p) & Em(p, a)',
                               effect='Em(c, a) & ~Dentro(c, p)',
                               domain='Carga(c) & Aviao(p) & Aeroporto(a)'),
                         Action('Voa(p, f, to)',
                               precond='Em(p, f)',
                               effect='Em(p, to) & ~Em(p, f)',
                               domain='Aviao(p) & Aeroporto(f) & Aeroporto(to)')
                     ],
                     domain='Carga(C1) & Carga(C2) & Aviao(P1) & Aviao(P2) & Aeroporto(LX) & Aeroporto(OPO)'
                    )

# Aplicar múltiplas ações para chegar à solução
solution = [expr("Carrega(C1 , P1, LX)"),
            expr("Voa(P1, LX, OPO)"),
            expr("Descarrega(C1, P1, OPO)"),
            expr("Carrega(C2, P2, OPO)"),
            expr("Voa(P2, OPO, LX)"),
            expr("Descarrega (C2, P2, LX)")] 

for action in solution:
    air_cargo.act(action)

print('Objetivo?',air_Cargo.goal_test())

Objetivo? False


In [106]:
import timeit

air_cargo = PlanningProblem(initial='Em(C1,LX) & Em(C2,OPO) & Em(P1,LX) & Em(P2,OPO)',
                     goals='Em(C1,OPO) & Em(C2,LX)',
                     actions=[
                         Action('Carrega(c, p, a)',
                               precond='Em(c, a) & Em(p, a)',
                               effect='Dentro(c, p) & ~Em(c, a)',
                               domain='Carga(c) & Aviao(p) & Aeroporto(a)'),
                         Action('Descarrega(c, p, a)',
                               precond='Dentro(c, p) & Em(p, a)',
                               effect='Em(c, a) & ~Dentro(c, p)',
                               domain='Carga(c) & Aviao(p) & Aeroporto(a)'),
                         Action('Voa(p, f, to)',
                               precond='Em(p, f)',
                               effect='Em(p, to) & ~Em(p, f)',
                               domain='Aviao(p) & Aeroporto(f) & Aeroporto(to)')
                     ],
                     domain='Carga(C1) & Carga(C2) & Aviao(P1) & Aviao(P2) & Aeroporto(LX) & Aeroporto(OPO)'
                    )

print('Forward:')
p = ForwardPlan(air_cargo)
start = timeit.default_timer()
air_cargo_solution = breadth_first_graph_search_plus(p, display=True).solution()
stop = timeit.default_timer()
air_cargo_solution = list(map(lambda action: Expr(action.name, *action.args), air_cargo_solution))
print('Solução:',air_cargo_solution)
print('Tempo de execução:', stop-start)

Forward:
224 paths have been expanded and 111 paths remain in the frontier
Solução: [Carrega(C1, P1, LX), Carrega(C2, P2, OPO), Voa(P1, LX, OPO), Descarrega(C1, P1, OPO), Voa(P2, OPO, LX), Descarrega(C2, P2, LX)]
Tempo de execução: 0.17773540000007415


## Exercício 9

Pretendemos comprar alguns artigos: um pacote de leite, bananas, e um livro. Estamos inicialmente em casa e sabemos que o leite e as bananas estão disponíveis no supermercado, e o livro numa livraria. 

a) Formule este problema como um problema de planeamento.

b) Obtenha os tempos de execução e os paths (activando o modo display na função *breadth_first_graph_search_plus*, `display=True`) no Forward.

In [96]:
# Resolução do exercicio 8

# SM = Supermarket; BS = Livroloja
shopping = PlanningProblem(initial='Em(Casa)',
                           goals='Tem(Leite) & Tem(Bananas) & Tem(Livro)',
                           actions=[
                               Action('Compra(x, loja)',
                                     precond='Em(loja)',
                                     effect='Tem(x)',
                                     domain='Loja(loja) & Item(x) & Vende(loja, x)'),
                               Action('Vai(x, y)',
                                     precond='Em(x)',
                                     effect='Em(y) & ~Em(x)',
                                     domain='Lugar(x) & Lugar(y)')],
                           domain='Lugar(Casa) & Lugar(SM) & Lugar(BS) & Loja(SM) & Loja(BS) & Item(Leite) & ' + \
                                  'Item(Bananas) & Item(Livro) & Vende(SM,Leite) & Vende(SM,Bananas) & Vende(BS,Livro)'
                          )

# Aplicar algumas ações
print('Objetivo?',shopping.goal_test())
shopping.act(expr('Vai(Casa,BS)'))
shopping.act(expr('Compra(Livro,BS)'))
shopping.act(expr('Vai(BS,SM)'))
shopping.act(expr('Compra(Bananas,SM)'))
shopping.act(expr('Compra(Leite,SM)'))
print('Objetivo?',shopping.goal_test())

Objetivo? False
Objetivo? True


In [98]:
import timeit
shopping = PlanningProblem(initial='Em(Casa)',
                           goals='Tem(Leite) & Tem(Bananas) & Tem(Livro)',
                           actions=[
                               Action('Compra(x, loja)',
                                     precond='Em(loja)',
                                     effect='Tem(x)',
                                     domain='Loja(loja) & Item(x) & Vende(loja, x)'),
                               Action('Vai(x, y)',
                                     precond='Em(x)',
                                     effect='Em(y) & ~Em(x)',
                                     domain='Lugar(x) & Lugar(y)')],
                           domain='Lugar(Casa) & Lugar(SM) & Lugar(BS) & Loja(SM) & Loja(BS) & Item(Leite) & ' + \
                                  'Item(Bananas) & Item(Livro) & Vende(SM,Leite) & Vende(SM,Bananas) & Vende(BS,Livro)'
                          )

print('Forward:')
p = ForwardPlan(shopping)
start = timeit.default_timer()
shopping_solution = breadth_first_graph_search_plus(p, display=True).solution()
stop = timeit.default_timer()
shopping_solution = list(map(lambda action: Expr(action.name, *action.args), shopping_solution))
print('Solução:',shopping_solution)
print('Tempo de execução:', stop-start)


Forward:
31 paths have been expanded and 9 paths remain in the frontier
Solução: [Vai(Casa, SM), Compra(Bananas, SM), Compra(Leite, SM), Vai(SM, BS), Compra(Livro, BS)]
Tempo de execução: 0.021835499999951935


## Exercício 10

CReplacesideremos uma torre de Hanoi com três discos (D1, D2 e D3), com buracos no centro e três pinos (A, B e C), Replacede os discos podem ser colocados. O disco D3 é maior que o disco D2, que é maior que o disco D1. Inicialmente, todos os discos estão no pino A, com D3 em baixo, D2 no meio e D1 no topo. Queremos movê-los para o pino C com a mesma cReplacefiguração (D3 em baixo, D2 no meio e D1 no topo). As seguintes regras aplicam-se:
* Apenas o disco do topo num pino pode ser deslocado
* Um disco não pode ser colocado por cima de um disco menor  

a) Formule este problema.
<img src="hanoi.png" width="200">

b) Obtenha os tempos de execução e os paths (activando o modo display na função *breadth_first_graph_search_plus*, `display=True`) no Forward.

In [90]:
# Resolução do exercício 9

estado = [
    expr('Sobre(D1,D2)'),
    expr('Sobre(D2,D3)'),
    expr('Sobre(D3,A)'),
    expr('Livre(D1)'),
    expr('Livre(B)'),
    expr('Livre(C)')
]

dominio = 'Disco(D1) & Disco(D2) & Disco(D3) & Pino(A) & Pino(B) & Pino(C) & Menor(D1,D2) & ' + \
            'Menor(D2,D3) & Menor(D1,D3) & Menor(D1,A) & Menor(D1,B) & Menor(D1,C) & ' + \
            'Menor(D2,A) & Menor(D2,B) & Menor(D2,C) & Menor(D3,A) & Menor(D3,B) & Menor(D3,C)'

goal = 'Sobre(D3,C) & Sobre(D2,D3) & Sobre(D1,D2)'

acoes = [Action('Move(d,x,y)', 
               precond='Livre(d) & Sobre(d,x) & Livre(y)',
               effect='Sobre(d,y) & Livre(x) & ~Sobre(d,x) & ~Livre(y)',
               domain='Disco(d) & Menor(d,x) & Menor(d,y)')]

th = PlanningProblem(initial=estado, goals=goal, actions=acoes, domain=dominio)

print(th.goal_test())
solution = [expr('Move(D1,D2,C)'),
            expr('Move(D2,D3,B)'),
            expr('Move(D1,C,D2)'),
            expr('Move(D3,A,C)'),
            expr('Move(D1,D2,A)'),
            expr('Move(D2,B,D3)'),
            expr('Move(D1,A,D2)')
           ]

for a in solution:
    th.act(a)
print('Objetivo?',th.goal_test())

False
Objetivo? True


In [91]:
th = PlanningProblem(initial=estado, goals=goal, actions=acoes, domain=dominio)

import timeit

print('Forward:')
p = ForwardPlan(th)
start = timeit.default_timer()
travel_solution = breadth_first_graph_search_plus(p).solution()
stop = timeit.default_timer()
travel_solution = list(map(lambda action: Expr(action.name, *action.args), travel_solution))
print('Solução:',travel_solution)
print('Tempo de execução:', stop-start)


Forward:
Solução: [Move(D1, D2, C), Move(D2, D3, B), Move(D1, C, D2), Move(D3, A, C), Move(D1, D2, A), Move(D2, B, D3), Move(D1, A, D2)]
Tempo de execução: 0.3650449000000435


In [92]:
# Outra solução mais simples: dominio e goal simplificados e com mais um esquema de ações
  # Resolução do exercício 9

estado = [
    expr('Sobre(D1,D2)'),
    expr('Sobre(D2,D3)'),
    expr('Sobre(D3,A)'),
    expr('Livre(D1)'),
    expr('Livre(B)'),
    expr('Livre(C)')
]

dominio = 'Disco(D1) & Disco(D2) & Disco(D3) & Pino(A) & Pino(B) & Pino(C) & Menor(D1,D2) & ' + \
            'Menor(D2,D3) & Menor(D1,D3) '

goal = 'Sobre(D3,C) & Sobre(D2,D3) & Sobre(D1,D2)'

acoes = [Action('MoveParaDisco(d,x,y)', 
               precond='Livre(d) & Sobre(d,x) & Livre(y)',
               effect='Sobre(d,y) & Livre(x) & ~Sobre(d,x) & ~Livre(y)',
               domain='Disco(d) & Disco(y)'),
        
         Action('MoveParaPino(d,x,y)', 
               precond='Livre(d) & Sobre(d,x) & Livre(y)',
               effect='Sobre(d,y) & Livre(x) & ~Sobre(d,x) & ~Livre(y)',
               domain='Disco(d) & Pino(y)')        
        ]



th = PlanningProblem(initial=estado, goals=goal, actions=acoes, domain=dominio)

print('Objectivo?',th.goal_test())
solution = [expr('MoveParaPino(D1,D2,C)'),
            expr('MoveParaPino(D2,D3,B)'),
            expr('MoveParaDisco(D1,C,D2)'),
            expr('MoveParaPino(D3,A,C)'),
            expr('MoveParaPino(D1,D2,A)'),
            expr('MoveParaDisco(D2,B,D3)'),
            expr('MoveParaDisco(D1,A,D2)')
           ]

for a in solution:
    th.act(a)
print('Objetivo?',th.goal_test())  

Objectivo? False
Objetivo? True


In [93]:
th = PlanningProblem(initial=estado, goals=goal, actions=acoes, domain=dominio)

import timeit

print('Forward:')
p = ForwardPlan(th)
start = timeit.default_timer()
travel_solution = breadth_first_graph_search_plus(p).solution()
stop = timeit.default_timer()
travel_solution = list(map(lambda action: Expr(action.name, *action.args), travel_solution))
print('Solução:',travel_solution)
print('Tempo de execução:', stop-start)


Forward:
Solução: [MoveParaPino(D1, D2, B), MoveParaDisco(D2, D3, D1), MoveParaPino(D3, A, C), MoveParaDisco(D2, D1, D3), MoveParaDisco(D1, B, D2)]
Tempo de execução: 0.20175180000001092
